In [13]:
from kedro.framework.context import load_context
proj_path = '../../../' 
context = load_context(proj_path)
# df = catalog.load("XXX")
parameters = context.params

In [24]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re

In [39]:
def exec_scraping_netkeiba_race_results(race_id_list):
    """
    netkeiba.comのレースIDのリストを渡して、それらをまとめて{'レースID', 結果のDataFrame}という形式の辞書型に格納する
    ex. race_results['201901010101']
        -> df 着順 枠番 馬番 馬名 性齢 斤量 騎手 タイム 着差 単勝 人気 馬体重 調教師 horse_id jockey_id course_id
    """
    race_results_dict = {}
    for race_id in race_id_list:
        try:
            # ベースの情報を取ってくる
            url = 'https://db.netkeiba.com/race/' + race_id
            race_results_dict[race_id] = pd.read_html(url)[0]
            
            # horse_idとjockey_idのために、再度対象URLのページを取得する
            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            # horse_idを取得する
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all("a", attrs={"href": re.compile("^/horse")})
            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])

            # jockey_idを取得する
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])

            # horse_idとjockey_idをベースに加える
            race_results_dict[race_id]["horse_id"] = horse_id_list
            race_results_dict[race_id]["jockey_id"] = jockey_id_list
      
            # course_idを追加
            race_results_dict[race_id]['course_id'] = [int(race_id[4:6])]*len(horse_id_list)
            
            time.sleep(0.1)

        except IndexError:
            continue
#         except:
#             break
    return race_results_dict

### mainの中身

In [40]:
race_id_list = []

for place in range(1,11):
    for kai in range(1,6):
        for day in range(1,9):
            for r in range(1,13):
                race_id = '2019' + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)
# -> ['201901010101', '201901010102', ,,, ]

# スクレイピング実行
race_results_dict = exec_scraping_netkeiba_race_results(race_id_list[0:parameters["scraping_limit"]])

In [41]:
race_results_dict

{'201901010101':    着順  枠番  馬番         馬名  性齢  斤量    騎手     タイム     着差     単勝  人気       馬体重  \
 0   1   1   1      ゴルコンダ  牡2  54  ルメール  1:48.3    NaN    1.4   1  518(-16)   
 1   2   3   3   プントファイヤー  牡2  54  岩田康誠  1:50.1      大    3.5   2   496(-8)   
 2   3   4   4  ラグリマスネグラス  牡2  51  団野大成  1:50.9      5   46.6   6   546(+6)   
 3   4   8   9     キタノコドウ  牡2  51  菅原明良  1:51.5  3.1/2   56.8   7   458(-8)   
 4   5   5   5   ネモフィラブルー  牡2  54  川島信二  1:51.7  1.1/2  140.3   9    436(0)   
 5   6   8   8  マイネルラクスマン  牡2  54  丹内祐次  1:52.1  2.1/2    9.7   3   480(+8)   
 6   7   2   2    サンモンテベロ  牝2  54   黛弘人  1:52.5  2.1/2  114.7   8   450(+2)   
 7   8   7   7   エスカレーション  牝2  54  藤岡佑介  1:52.5    アタマ   26.1   5   448(-4)   
 8   9   6   6   セイウンジュリア  牝2  54  池添謙一  1:52.6     クビ   16.4   4    470(0)   
 
         調教師    horse_id jockey_id  course_id  
 0  [東] 木村哲也  2017105318     05339          1  
 1  [東] 手塚貴久  2017104612     05203          1  
 2  [東] 藤沢和雄  2017103879     01180          1  


In [42]:
# 表として見やすいように、dfのindexにrace idを入れる
for key in race_results_dict.keys():
    race_results_dict[key].index = [key]*len(race_results_dict[key])

# 各レース結果のdfを1つに結合する
race_results_df = pd.concat((race_results_dict[key] for key in race_results_dict.keys()),sort=False)

In [43]:
race_results_df

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id,course_id
201901010101,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也,2017105318,05339,1
201901010101,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久,2017104612,05203,1
201901010101,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄,2017103879,01180,1
201901010101,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登,2017106259,01179,1
201901010101,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人,2017104140,01062,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201901010208,3,4,4,エンシュラウド,セ3,54,ルメール,1:48.6,1.1/2,2,1.0,470(-4),[東] 藤沢和雄,2016110030,05339,1
201901010208,4,2,2,グランツリヒター,牡3,54,三浦皇成,1:49.0,2.1/2,12.3,4.0,496(-8),[東] 中川公成,2016103280,01122,1
201901010208,5,7,7,エピローグ,牝3,52,荻野極,1:49.4,2.1/2,19.5,5.0,458(-8),[西] 小崎憲,2016101402,01160,1
201901010208,6,3,3,サイモンルチアーノ,牡4,57,黛弘人,1:49.9,3,147.5,7.0,492(+6),[東] 田中清隆,2015109012,01109,1
